## sentencepiece
- google에서 개발한 하위 단어 토크나이저 라이브러리
- 바이트 페어 인코딩, 워드피스, 유니코드  등 다양한 알고리즘 지원
- 사용자가 직접 설정할 수 있는 하이퍼파라미터 제공

In [ ]:
!pip install sentencepiece Korpora

### 1. data : 청화대 청원 게시판에 올라온 청원 말뭉치

In [ ]:
from Korpora import Korpora

corpus = Korpora.load("korean_petitions")

### 학습 데이터 확인

In [2]:
dataset = corpus.train
print('dataset 수:', len(dataset))


dataset 수: 433631


In [3]:
petition = dataset[0]
print("청원 시작일:", petition.begin)
print("청원 종료일:", petition.end)
print("청원 동의 수:", petition.num_agree)
print("청원 제목:", petition.title)
print("청원 본문:", petition.text[:30])

청원 시작일: 2017-08-25
청원 종료일: 2017-09-24
청원 동의 수: 88
청원 제목: 학교는 인력센터, 취업센터가 아닙니다. 정말 간곡히 부탁드립니다.
청원 본문: 안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비


### 학습 데이터 생성


In [16]:
petitions = corpus.get_all_texts()
print(petitions[0])

안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용, 기간제 교사, 영전강, 스강의 무기계약직화가 그들의 임용 절벽과는 전혀 무관한 일이라고 주장하고 있지만 조금만 생각해보면 전혀 설득력 없는 말이라고 생각합니다. 학교 수가 같고, 학생 수가 동일한데 영양교사와 기간제 교사, 영전강 스강이 학교에 늘어나게 되면 당연히 정규 교원의 수는 줄어들게 되지 않겠습니까? 기간제 교사, 영전강, 스강의 무기계약직화, 정규직화 꼭 전면 백지화해주십시오. 백년대계인 국가의 교육에 달린 문제입니다. 단순히 대통령님의 일자리 공약, 81만개 일자리 창출 공약을 지키시고자 돌이킬 수 없는 실수는 하지 않으시길 바랍니다. 세계 어느 나라와 비교해도, 한국 교원의 수준과 질은 최고 수준입니다. 고등교육을 받고 어려운 국가 고시를 통과해야만 대한민국 공립 학교의 교단에 설 수 있고, 이러한 과정이 힘들기는 하지만 교원들이 교육자로서의 사명감과 자부심을 갖고 교육하게 되는 원동력이기도 합니다. 자격도 없는 비정규 인력들을 일자리 늘리기 명목 하에 학교로 들이게 되면, 그들이 무슨 낯으로 대한민국이 '공정한 사회' 라고 아이들에게 가르칠 수 있겠습니까? 그들이 가르치는 것을 학부모와 학생들이 납득할 수 있겠으며, 학생들은 공부를 열심히 해야하는 이유를 찾을 수나 있겠습니까? 열심히 안 해도 떼 쓰면 되는 세상이라고 생각하지 않겠습니까? 영양사의 영양교사화도 재고해주십시오. 영양사분들 정말 너무나 고마운 분들입니다. 학생들의 건강과 영양? 당연히 성장기에 있는 아이들에게 필수적이고 중요한 문제입니다. 하지만 이들이 왜 교사입니까. 유래를 찾아 볼 수 없는 영양사의 '교사'화. 정말 대통령님이 생각하신 아이디어라고 믿기 싫을 정도로 납득하기 어렵습니다. 중등은 실과교과 교사가 존재하지요? 초등 역시 임용 시험에 실과가 포함돼 있으며 학교 현장에서도 정규 교원이 직접 

In [7]:

corpus_file = "./datasets/corpus.txt"
with open(corpus_file, "w", encoding = "utf-8") as f:
    for petition in petitions:
        f.write(petition + "\n")

## 2. 토크 나이저 모델 학습
<span style="color:red">
- petition_bep.model : 학습된 토크나이저 파일<br>
- petition_vocab : 어휘 사전 파일
  </span>

In [17]:
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(
    input = corpus_file,              # 말뭉치 텍스트 파일 경로
    model_prefix = "1.petition_bep",      # 모델 파일 이름
    vocab_size = 8000,                # 어휘 사전 크기
    model_type = "bpe",               # 토크나이저 알고리즘 (unigram, bpe, char, word)
    
)

## 3.토큰화(.model 이용)

In [22]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("1.petition_bep.model")

True

In [36]:
print("pad[0]: ", tokenizer.pad_id())
print("unkonwn[1]: ", tokenizer.unk_id())
print("begin of sequence[2]: ", tokenizer.bos_id())
print("end of sequence[3]: ", tokenizer.eos_id())



pad[0]:  -1
unkonwn[1]:  0
begin of sequence[2]:  1
end of sequence[3]:  2


### 토큰화

In [23]:
sentence = "안녕하세요, 토크나이저를 학습했습니다."
sentences = ["안녕하세요", "토크나이저는 자연어 전처리 방법입니다.", "다음은 임베딩입니다."]

tokenized_sentence = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = tokenizer.encode_as_pieces(sentences)

print(sentence, "->" , tokenized_sentence)
print(sentences, "->" , tokenized_sentences)


안녕하세요, 토크나이저를 학습했습니다. -> ['▁안녕하세요', ',', '▁토', '크', '나', '이', '저', '를', '▁학', '습', '했습니다', '.']
['안녕하세요', '토크나이저는 자연어 전처리 방법입니다.', '다음은 임베딩입니다.'] -> [['▁안녕하세요'], ['▁토', '크', '나', '이', '저는', '▁자연', '어', '▁전', '처리', '▁방법', '입니다', '.'], ['▁다음', '은', '▁임', '베', '딩', '입니다', '.']]


##### 밑줄 문자(underscope) : U+2581
- 문장의 첫글자
- 띄어쓰기, 공백

### 인코딩, 디코딩

In [41]:
encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = tokenizer.encode_as_ids(sentences)

print(tokenized_sentence, "->" , encoded_sentence)
print(tokenized_sentences, "->" , encoded_sentences)
print(len(tokenized_sentence), "->" , len(encoded_sentence))
print(len(tokenized_sentences[1]), "->" , len(encoded_sentences[1]))

['▁안녕하세요', ',', '▁토', '크', '나', '이', '저', '를', '▁학', '습', '했습니다', '.'] -> [667, 6553, 994, 6880, 6544, 6513, 6590, 6536, 110, 6554, 579, 6515]
[['▁안녕하세요'], ['▁토', '크', '나', '이', '저는', '▁자연', '어', '▁전', '처리', '▁방법', '입니다', '.'], ['▁다음', '은', '▁임', '베', '딩', '입니다', '.']] -> [[667], [994, 6880, 6544, 6513, 4019, 2118, 6539, 49, 2467, 712, 23, 6515], [1087, 6532, 249, 7068, 7358, 23, 6515]]
12 -> 12
12 -> 12


In [44]:
decoded_sentence = tokenizer.decode_ids(encoded_sentence)
decoded_sentences = tokenizer.decode_ids(encoded_sentences)

print(encoded_sentence, "->" , decoded_sentence)
print(encoded_sentences, "->" , decoded_sentences)


[667, 6553, 994, 6880, 6544, 6513, 6590, 6536, 110, 6554, 579, 6515] -> 안녕하세요, 토크나이저를 학습했습니다.
[[667], [994, 6880, 6544, 6513, 4019, 2118, 6539, 49, 2467, 712, 23, 6515], [1087, 6532, 249, 7068, 7358, 23, 6515]] -> ['안녕하세요', '토크나이저는 자연어 전처리 방법입니다.', '다음은 임베딩입니다.']


## 4. 어휘 사전

In [45]:
vocab = {idx:tokenizer.id_to_piece(idx) for idx in range(tokenizer.get_piece_size())}

In [47]:
print("vocab_size: ", len(vocab))

vocab_size:  8000


In [48]:
print(list(vocab.items())[:5])

[(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '니다'), (4, '▁이')]


- 0: OOV에 해당하는 단어
- \<s> : 문장의 시작 지점
- \</s> : 문장의 종료 지점

<span style="color:blue">3번에서는 문장 첫 시작에 underscope로 나왔지?</span>

<Reference>
- 파이토치 트랜스포머를 활용한 자연어 처리와 컴퓨터비전 심층학습(위키북스, 윤대희, 김동화, 송종민, 진현두 지음, 2023)<br>
